In [1]:
import math
import pandas as pd
import pyarrow
import fastparquet 

import os
import configparser
import datetime as dt
from datetime import datetime
from os.path import exists
import time
from pytimeparse.timeparse import timeparse

import folium
from random import randint

import shapely
from shapely.geometry import Point, Polygon, LineString, GeometryCollection
from shapely.ops import nearest_points
from pyproj import Geod
import numpy as np

In [2]:
# Method to read config file settings
def read_config(Config_File):
    config = configparser.ConfigParser()
    config.read(Config_File)
    return config

configurations = read_config("configurations.ini")

In [3]:
# LOAD NEEDED DATA

In [4]:
# Load RT data (gtfs_data)

In [5]:
rt_gtfs_data = pd.read_parquet('data\kh_transitview.parquet')
rt_gtfs_data = rt_gtfs_data.sort_values(by = ['file'])
rt_gtfs_data.head(5)

,BlockID,Direction,Offset,Offset_sec,VehicleID,destination,estimated_seat_availability,file,heading,label,lat,late,lng,next_stop_id,next_stop_name,next_stop_sequence,original_late,route,timestamp,trip
0,1081,SouthBound,1,10,8664,Frankford Transportation Center,NOT_AVAILABLE,2022-09-01,210,8664,40.093037,3,-75.017159,21945.0,Roosevelt Blvd & Plaza Dr - FS,71.0,3,14,2022-09-01 04:00:04+00:00,87728
128,5391,WestBound,1,7,7381,35th-Allegheny,NOT_AVAILABLE,2022-09-01,270,7381,40.001133,-2,-75.148354,20599.0,Allegheny Av & 5th St,27.0,-2,60,2022-09-01 04:00:04+00:00,75616
129,5436,NorthBound,1,27,7403,27th-Allegheny,NOT_AVAILABLE,2022-09-01,360,7403,39.978828,1,-75.182526,14942.0,29th St & Thompson St,44.0,2,48,2022-09-01 04:00:04+00:00,70324
130,5439,SouthBound,0,4,7412,Front-Market,NOT_AVAILABLE,2022-09-01,105,7412,39.949955,0,-75.142204,NaN,None,NaN,-2,48,2022-09-01 04:00:04+00:00,70182
131,5441,SouthBound,1,22,7319,Front-Market,NOT_AVAILABLE,2022-09-01,180,7319,39.989803,-1,-75.180038,30270.0,29th St & Huntingdon St,8.0,-1,48,2022-09-01 04:00:04+00:00,70183


In [6]:
rt_gtfs_data.dtypes

BlockID                                      int64
Direction                                   object
Offset                                       int64
Offset_sec                                   int64
VehicleID                                    int64
destination                                 object
estimated_seat_availability                 object
file                                datetime64[ns]
heading                                      int64
label                                        int64
lat                                        float64
late                                         int64
lng                                        float64
next_stop_id                               float64
next_stop_name                              object
next_stop_sequence                         float64
original_late                                int64
route                                       object
timestamp                      datetime64[ns, UTC]
trip                           

In [7]:
# Load Schedule Trips Data (trips_data)

In [8]:
sched_folder = 'sch_gtfs\\schedule_v202209101'

In [9]:
trips_f = sched_folder + '\\' + 'trips.txt'
trips_data = pd.read_csv(trips_f)
trips_data.head()

,route_id,service_id,trip_id,trip_headsign,block_id,direction_id,shape_id
0,1,10,87152,Decatur-Drummond,1003,0,275147
1,1,10,87153,Parx Casino,2151,0,275145
2,1,10,87154,Parx Casino via Decatur-Drummond,2155,0,275152
3,1,10,87155,Decatur-Drummond,1426,0,275151
4,1,10,87156,Parx Casino via Decatur-Drummond,2153,0,275152


In [10]:
# Load Schedule Shapes Data (shapes_data)

In [11]:
shapes_f = sched_folder + '\\' + 'shapes.txt'
shapes_data = pd.read_csv(shapes_f)
shapes_data.head()

,shape_id,shape_pt_lat,shape_pt_lon,shape_pt_sequence
0,275145,40.105486,-75.000047,1072
1,275145,40.110772,-74.992364,1097
2,275145,40.112407,-74.989999,1104
3,275145,40.052940,-75.049911,953
4,275145,40.011494,-75.200398,467


In [12]:
# Load Schedule stops Data (stops_data)

In [13]:
stops_f = sched_folder + '\\' + 'stop_times.txt'
stops_data = pd.read_csv(stops_f)
stops_data.head()

,trip_id,arrival_time,departure_time,stop_id,stop_sequence
0,33584,07:14:00,07:14:00,31294,1
1,33584,07:14:00,07:14:00,20610,2
2,33584,07:15:00,07:15:00,20611,3
3,33584,07:16:00,07:16:00,20612,4
4,33584,07:18:00,07:18:00,20613,5


In [14]:
###############################################################################
###############################################################################
###############################################################################

In [15]:
# Function to return schedule start/end times and first/last stop_ids of a trip_id
def get_sched_stop_times(trip_id):
    
    data = stops_data.loc[stops_data['trip_id'] == trip_id]
    
    if not data.empty:
        data = data.sort_values(by=['stop_sequence'])
        data1 = data.iloc[0]
        data2 = data.iloc[-1]
    
        return [data1['departure_time'], data1['stop_id'], data1['stop_sequence'],\
                data2['arrival_time'], data2['stop_id'], data2['stop_sequence']]
    
    else:
        return ['NA']

In [16]:
# Function to return schedule shape of a shape_id
def get_sched_shape(shape_id):
    
    data = shapes_data.loc[shapes_data['shape_id'] == shape_id]
    data = data.sort_values(by=['shape_pt_sequence'])
    data = list(zip(data.shape_pt_lat, data.shape_pt_lon))
    
    if len(data) > 1:
        return data
    else:
        return ['NA']

In [17]:
# Function to return schedule route_id, trip_id, shape_id, shapes for a given route list
def get_sched_route_gtfs(routes):

    data = pd.DataFrame(columns=['route_id', 'trip_id', 'sch_shape_id', 'sch_dep_time', 'sch_first_stop_id',
                                 'sch_first_stop_seq', 'sch_arr_time','sch_last_stop_id', 'sch_last_stop_seq',
                                 'sch_shape', 'rt_shape'])
    
    i = 0 #output DF index
    
    for route_id in routes:
        # for route, get all trips and keep needed columns
        trip = trips_data.loc[trips_data['route_id'] == route_id]
        trip = trip[['route_id', 'trip_id', 'shape_id']]

        # iterate over trips
        for index, row in trip.iterrows():
            # get trip stop and shape information
            stops = get_sched_stop_times(row['trip_id'])
            sched_shape = get_sched_shape(row['shape_id'])
            
            # if stop and shape info found add trip to output
            if stops[0] != 'NA' and sched_shape[0] != 'NA':
                
                data.at[i, 'route_id'] = row['route_id']
                data.at[i, 'trip_id'] = row['trip_id']
                data.at[i, 'sch_shape_id'] = row['shape_id']
                data.at[i, 'sch_dep_time'] = stops[0]
                data.at[i, 'sch_first_stop_id'] = stops[1]
                data.at[i, 'sch_first_stop_seq'] = stops[2]
                data.at[i, 'sch_arr_time'] = stops[3]
                data.at[i, 'sch_last_stop_id'] = stops[4]
                data.at[i, 'sch_last_stop_seq'] = stops[5]
                
                data.at[i, 'sch_shape'] = sched_shape
                
                i += 1
                
    data = data.astype({"route_id":'str', "trip_id":'Int64', "sch_shape_id":'Int64', "sch_dep_time":'str',
                        "sch_first_stop_id":'Int64', "sch_first_stop_seq":'Int64', "sch_arr_time":'str',
                        "sch_last_stop_id":'Int64', "sch_last_stop_seq":'Int64'}) 
        
    return data

In [18]:
def get_rt_shape(data):

    shape = data.sort_values(by = ['file'])
    shape = list(zip(shape.lat, shape.lng))
    
    # only return RT shapes that have more than 1 point
    if len(shape) > 1:
        return shape
    else:
        return 'NA'

In [19]:
def get_rt_gtfs(trip_id, sched_dep_dt, sched_first_stop_id, sched_arr_dt, sch_last_stop_id):
    
    # WILL EXCLUDE ALL TRIPS THAT DON'T START AT SCHEDULED STOP ID
    
    # look for RT data starting 5 minutes before sched departure, and arriving max 3 hours after sched arrival
    minus_delta = dt.timedelta(minutes=5)
    plus_delta = dt.timedelta(hours=3)
    
    data = rt_gtfs_data.loc[(rt_gtfs_data['file'] >= sched_dep_dt - minus_delta) & (rt_gtfs_data['file'] <= sched_arr_dt + plus_delta) &\
                            (rt_gtfs_data['trip'] == trip_id)]
    # exclude RT trips that have only 1 coordinate 
    if len(data.index) >= 2:
        data = data.sort_values(by=['file'])
        
        # drop all early feed updates where next stop id = sched_first_stop_id, except last one
        # this should eliminate feed updates before trip starts
        prev_index = -9        
        for index, row in data.iterrows():
            if row['next_stop_id'] == sched_first_stop_id and prev_index == -9:
                prev_index = index
            elif row['next_stop_id'] == sched_first_stop_id and prev_index != -9:
                data = data.drop([prev_index])
                prev_index = index
            else:
                break
                
        # if route still has more than 1 coordinate
        if len(data.index) >= 2:
            # keep only one feed update after last stop where next stop id is NaN
            # this should drop feed updates after bus arrives at last scheduled stop
            for index, row in data.iterrows():
                if row['next_stop_id'] == sch_last_stop_id:
                    if len(data.index) > index + 1:
                        if math.isnan(data.iloc[index+1]['next_stop_id']):
                            data = data.loc[:index+1]
            
#            while (math.isnan(data.iloc[-1]['next_stop_id'])):
#                if math.isnan(data.iloc[-2]['next_stop_id']):
#                    data = data[:-1]
#                else:
#                    break

            # if more than 1 coordinate in trip remains then
            if len(data.index) >= 2:
                # take first and before last feed updates to get first/last dep/arr info
                data1 = data.iloc[0] 
                data2 = data.iloc[-2]

                # get the RT shape
                rt_shape = get_rt_shape(data)

                if rt_shape != 'NA':
                    return[data1['Direction'], data1['VehicleID'], data1['file'], data1['next_stop_id'], data1['next_stop_sequence'],
                                                                   data2['file'], data2['next_stop_id'], data2['next_stop_sequence'], rt_shape]
                else:
                    return['NA']
            else:
                return['NA']
        else:
            return['NA']
    else:
        return['NA']  

In [20]:
def make_dataset(routes, dates):
    
    data = get_sched_route_gtfs(routes) 
    
    out_data = pd.DataFrame()
    
    # get date at start and of search window
    from_dt = datetime.strptime(dates[0], '%m/%d/%Y %H:%M')
    from_day = from_dt.date()
    to_dt = datetime.strptime(dates[1], '%m/%d/%Y %H:%M')
    to_day = to_dt.date()
    day_delta = dt.timedelta(days=1)
    
    # add columns to sched GTFS DF
    data = data.assign(rt_direction = np.nan,
                       rt_vehicle_id = np.nan,
                       rt_dep_time = np.nan,
                       rt_first_stop_id = np.nan,
                       rt_first_stop_seq = np.nan,
                       rt_arr_time = np.nan,
                       rt_last_stop_id = np.nan,
                       rt_last_stop_seq = np.nan,
                      )
    
    #get trips on each day between from_day -> to_day
    while (from_day <= to_day):
        
        # making a copy bec same trip id may exist on multiple dates being retrieved 
        day_data = data.copy()
        
        # for each trip_id on this day
        for index, row in day_data.iterrows():
            
            # create datetime of scheduled departure and arrival
            sched_dep_dt = datetime.combine(from_day, dt.time.min) + dt.timedelta(seconds = timeparse(row['sch_dep_time']))
            sched_arr_dt = datetime.combine(from_day, dt.time.min) + dt.timedelta(seconds = timeparse(row['sch_arr_time']))  
            
            # if trip is scheduled between requested datetimes
            if sched_dep_dt >= from_dt and sched_dep_dt <= to_dt:
                
                rt_data = get_rt_gtfs(row['trip_id'], sched_dep_dt, row['sch_first_stop_id'], sched_arr_dt, row['sch_last_stop_id'])
                
                # if RT gtfs data found for this day
                if rt_data[0] != 'NA': 
                    day_data.at[index, 'rt_direction'] = rt_data[0]
                    day_data.at[index, 'rt_vehicle_id'] = rt_data[1]
                    day_data.at[index, 'sch_dep_time'] = sched_dep_dt
                    day_data.at[index, 'sch_arr_time'] = sched_arr_dt
                    day_data.at[index, 'rt_dep_time'] = rt_data[2]
                    day_data.at[index, 'rt_first_stop_id'] = rt_data[3]
                    day_data.at[index, 'rt_first_stop_seq'] = rt_data[4]
                    day_data.at[index, 'rt_arr_time'] = rt_data[5]
                    day_data.at[index, 'rt_last_stop_id'] = rt_data[6]
                    day_data.at[index, 'rt_last_stop_seq'] = rt_data[7]
                    day_data.at[index, 'rt_shape'] = rt_data[8]
                
                # if no RT gtfs found for this day drop today's trip 
                else:
                    day_data = day_data.drop([index])
            # if scheduled trip departure time out of window drop today's trip     
            else:
                day_data = day_data.drop([index])
                
        out_data = pd.concat([out_data, day_data])
        
        from_day += day_delta
    
    # set data types
    out_data = out_data.astype({"route_id":'str', "trip_id":'Int64', "sch_shape_id":'Int64', "rt_direction":'str', "rt_vehicle_id":'Int64',
                                "sch_dep_time":'datetime64[ns]', "sch_first_stop_id":'Int64', "sch_first_stop_seq":'Int64',
                                "sch_arr_time":'datetime64[ns]', "sch_last_stop_id":'Int64', "sch_last_stop_seq":'Int64',
                                "rt_dep_time":'datetime64[ns]', "rt_first_stop_id":'Int64', "rt_first_stop_seq":'Int64',
                                "rt_arr_time":'datetime64[ns]', "rt_last_stop_id":'Int64', "rt_last_stop_seq":'Int64'}) 
    
    # reorder columns
    out_data = out_data[['route_id', 'trip_id', 'sch_shape_id', 'rt_direction', 'rt_vehicle_id', 'sch_dep_time', 'sch_first_stop_id',
                         'sch_first_stop_seq', 'sch_arr_time', 'sch_last_stop_id', 'sch_last_stop_seq', 'rt_dep_time', 'rt_first_stop_id',
                         'rt_first_stop_seq', 'rt_arr_time', 'rt_last_stop_id', 'rt_last_stop_seq', 'sch_shape', 'rt_shape']]
    
    # reset the DF index
    out_data = out_data.reset_index(drop = True)
    
    return out_data 

In [21]:
# PREPARE SCHEDULE + RT GTFS INTEGRATED DATASET

#routes = list(rt_gtfs_data.route.unique())
routes = ['12', '57']
start_between = ['9/01/2022 00:00', '9/01/2022 23:59']

data_set = make_dataset(routes, start_between) 

data_set.head()

,route_id,trip_id,sch_shape_id,rt_direction,rt_vehicle_id,sch_dep_time,sch_first_stop_id,sch_first_stop_seq,sch_arr_time,sch_last_stop_id,sch_last_stop_seq,rt_dep_time,rt_first_stop_id,rt_first_stop_seq,rt_arr_time,rt_last_stop_id,rt_last_stop_seq,sch_shape,rt_shape
0,12,57867,275161,EastBound,8045,2022-09-01 10:33:00,317,1,2022-09-01 11:07:00,32259,49,2022-09-01 10:38:00,317,1,2022-09-01 11:15:00,<NA>,<NA>,"[(39.940038, -75.213789), (39.939972, -75.2138...","[(39.940384, -75.210617), (39.940689, -75.2029..."
1,12,57868,275161,EastBound,8027,2022-09-01 11:14:00,317,1,2022-09-01 11:47:00,32259,49,2022-09-01 11:16:00,317,1,2022-09-01 11:58:00,32259,49,"[(39.940038, -75.213789), (39.939972, -75.2138...","[(39.940029, -75.21111299999998), (39.941044, ..."
2,12,57870,275161,EastBound,3101,2022-09-01 11:53:00,317,1,2022-09-01 12:28:00,32259,49,2022-09-01 11:55:00,317,1,2022-09-01 12:38:00,<NA>,<NA>,"[(39.940038, -75.213789), (39.939972, -75.2138...","[(39.940521, -75.210403), (39.940632, -75.2026..."
3,12,57871,275161,EastBound,8033,2022-09-01 12:13:00,317,1,2022-09-01 12:48:00,32259,49,2022-09-01 12:15:00,317,1,2022-09-01 12:58:00,<NA>,<NA>,"[(39.940038, -75.213789), (39.939972, -75.2138...","[(39.940578, -75.21036499999998), (39.941238, ..."
4,12,57872,275161,EastBound,8040,2022-09-01 12:33:00,317,1,2022-09-01 13:08:00,32259,49,2022-09-01 12:35:00,317,1,2022-09-01 13:18:00,<NA>,<NA>,"[(39.940038, -75.213789), (39.939972, -75.2138...","[(39.939919, -75.21125), (39.940689, -75.20293..."


In [22]:
data_set.dtypes

route_id                      object
trip_id                        Int64
sch_shape_id                   Int64
rt_direction                  object
rt_vehicle_id                  Int64
sch_dep_time          datetime64[ns]
sch_first_stop_id              Int64
sch_first_stop_seq             Int64
sch_arr_time          datetime64[ns]
sch_last_stop_id               Int64
sch_last_stop_seq              Int64
rt_dep_time           datetime64[ns]
rt_first_stop_id               Int64
rt_first_stop_seq              Int64
rt_arr_time           datetime64[ns]
rt_last_stop_id                Int64
rt_last_stop_seq               Int64
sch_shape                     object
rt_shape                      object
dtype: object

In [23]:
def cleaning_anal(data):
    
    for index, row in data.iterrows():
        
        sch_ln = LineString(row['sch_shape'])
        rt_ln = LineString(row['rt_shape'])
        
        # length ratios of RT to scheduled shape
        sch_len = LineString(sch_ln).length
        rt_len = LineString(rt_ln).length
        
        data.at[index, 'rt_sch_len_ratio'] = round(rt_len/sch_len, 3)
        
        # max deviation distance
        h_dist = rt_ln.hausdorff_distance(sch_ln)
                
        data.at[index, 'max_detour_dist'] = h_dist

    return data

In [24]:
data_set_cl = cleaning_anal(data_set)
data_set_cl = data_set_cl.sort_values(by=['max_detour_dist'])
pd.concat([data_set_cl.head(3), data_set_cl.tail(3)])

,route_id,trip_id,sch_shape_id,rt_direction,rt_vehicle_id,sch_dep_time,sch_first_stop_id,sch_first_stop_seq,sch_arr_time,sch_last_stop_id,...,rt_dep_time,rt_first_stop_id,rt_first_stop_seq,rt_arr_time,rt_last_stop_id,rt_last_stop_seq,sch_shape,rt_shape,rt_sch_len_ratio,max_detour_dist
158,57,73892,275578,SouthBound,8271,2022-09-01 17:23:00,844,33,2022-09-01 18:21:00,99,...,2022-09-01 17:33:00,844,33,2022-09-01 18:57:00,<NA>,<NA>,"[(40.034178, -75.115393), (40.03376, -75.11549...","[(40.033058, -75.116074), (40.030613, -75.1181...",0.969,0.001311
87,12,57971,275162,WestBound,8028,2022-09-01 17:55:00,32259,1,2022-09-01 18:29:00,317,...,2022-09-01 17:58:00,32259,1,2022-09-01 18:48:00,<NA>,<NA>,"[(39.94526, -75.143032), (39.945265, -75.14311...","[(39.945309, -75.142914), (39.945446, -75.1437...",0.984,0.001391
60,12,57939,275162,WestBound,8028,2022-09-01 21:00:00,32259,1,2022-09-01 21:26:00,317,...,2022-09-01 21:02:00,32259,1,2022-09-01 21:55:00,<NA>,<NA>,"[(39.94526, -75.143032), (39.945265, -75.14311...","[(39.945446, -75.143806), (39.945198, -75.1499...",0.976,0.001411
209,57,73943,275598,NorthBound,8639,2022-09-01 22:53:00,24973,1,2022-09-01 23:45:00,841,...,2022-09-01 22:59:00,24973,1,2022-09-01 23:21:00,17607,57,"[(39.913685, -75.15574), (39.913756, -75.15619...","[(39.914589, -75.15589099999998), (39.916737, ...",0.187,0.105855
125,57,73856,275580,SouthBound,8639,2022-09-01 21:34:00,841,1,2022-09-01 22:30:00,99,...,2022-09-01 22:25:00,18100,97,2022-09-01 22:53:00,<NA>,<NA>,"[(40.041995, -75.136952), (40.04353, -75.13665...","[(39.942833, -75.148888), (39.942833, -75.1488...",0.149,0.113355
149,57,73883,275581,SouthBound,8639,2022-09-01 15:54:00,841,1,2022-09-01 17:16:00,24981,...,2022-09-01 16:05:00,17932,11,2022-09-01 16:23:00,17965,40,"[(40.041995, -75.136952), (40.04353, -75.13665...","[(40.050358, -75.12258099999998), (40.052807, ...",0.112,0.134952


In [25]:
index_to_plot = 149

map = folium.Map(location=[40,-75.154839], zoom_start= 12)
  
folium.PolyLine(data_set_cl.iloc[index_to_plot]['sch_shape'], color = 'blue', popup = 'Schedule shape', weight=7, opacity=0.6).add_to(map) 
folium.PolyLine(data_set_cl.iloc[index_to_plot]['rt_shape'], color = 'red', popup = 'RT shape', weight=7, opacity=0.6).add_to(map) 

map

In [26]:
#def cleaning_anal(data):
#    
#    for index, row in data.iterrows():
#        
#        # length ratios of RT to scheduled shape
#        sch_shape = row['sch_shape']
#        rt_shape = row['rt_shape']
#        
#        geod = Geod(ellps = "WGS84")
#        sch_len = geod.geometry_length(LineString(sch_shape))
#        rt_len = geod.geometry_length(LineString(rt_shape))
#        
#        data.at[index, 'rt_sch_len_ratio'] = round(rt_len/sch_len, 3)
#        
#        # max deviation distance
#        max_dist = 0.
#        for rt_pt in rt_shape:
#            
#            min_dist = float('inf')
#            for sch_pt in sch_shape:
#                _,_,dist = geod.inv(rt_pt[1], rt_pt[0], sch_pt[1], sch_pt[0])
#                if dist < min_dist:
#                    min_dist = dist
#            
#            if min_dist > max_dist:
#                max_dist = min_dist
#                
#        data.at[index, 'max_detour_dist_km'] = max_dist
#
#    return data